In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

TC_LABELS_FILE = '../data/train-task2-TC-with-spans.labels'
SI_LABELS_FILE = '../data/train-data-with-sents-no-maxlen.tsv'
SI_LABELS_FILE_BASELINE = '../data/train-data-with-sents-baseline.tsv'
SI_LABELS_FILE_IMPROVED = '../data/train-data-with-sents-improved.tsv'

# TC (Task 2)

In [2]:
tc_cols = {"doc_id": int, "label": str, "idx_start": int, "idx_end": int, "sent": str}
# Somehow, the pandas + UTF-8 combination doesn't work for this file even though UTF-8 worked for generating this file...
# The Latin encoding it is then. 
# The quoting=3 part ('treat quotes as normal characters') is important!!
df = pd.read_csv(TC_LABELS_FILE, sep='\t', names=tc_cols.keys(), encoding='ISO-8859-1', quoting=3)
df = df.astype(tc_cols)
df['length_words'] = df['sent'].apply(lambda x : len(str(x).split()))
df['length_chars'] = df['idx_end'] - df['idx_start']
df.head()

,doc_id,label,idx_start,idx_end,sent,length_words,length_chars
0,111111111,Appeal_to_Authority,265,323,The next transmission could be more pronounced...,9,58
1,111111111,Appeal_to_Authority,1795,1935,when (the plague) comes again it starts from m...,26,140
2,111111111,Doubt,149,157,appeared,1,8
3,111111111,Repetition,1069,1091,"a very, very different",4,22
4,111111111,Appeal_to_fear-prejudice,1334,1462,He also pointed to the presence of the pneumon...,22,128


In [3]:
df.groupby(['label']).length_words.describe().sort_values("count", ascending=False)

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Loaded_Language,2200.0,3.819091,4.491921,1.0,2.0,3.0,5.0,90.0
"Name_Calling,Labeling",1105.0,3.933032,3.351001,1.0,2.0,3.0,5.0,28.0
Repetition,621.0,2.813205,3.453454,1.0,1.0,2.0,3.0,31.0
Doubt,496.0,21.122984,16.078815,1.0,9.0,17.0,29.0,141.0
"Exaggeration,Minimisation",493.0,7.582150,5.863266,1.0,3.0,6.0,10.0,43.0
Appeal_to_fear-prejudice,321.0,16.803738,13.106372,1.0,6.0,13.0,25.0,74.0
Flag-Waving,250.0,10.824000,11.614327,1.0,2.0,7.0,15.0,73.0
Causal_Oversimplification,212.0,21.471698,12.544297,3.0,13.0,19.0,28.0,71.0
Appeal_to_Authority,155.0,22.458065,21.430419,2.0,9.0,16.0,28.5,131.0


In [4]:
pd.set_option('display.max_colwidth', -1)
df[df.length_words > 60][['label', 'sent']]

,label,sent
151,Appeal_to_Authority,"Edward E. Curtis IV, the author of Black Muslim Religion in the Nation of Islam, said that while estimates ranged wildly about how many bona fide members belonged to the Nation, he believes that there were tens of thousands of members in the 1960s and early 1970s. But he said that millions of others sympathized with the movements anticolonial stance and were inspired by its most famous member, Muhammad Ali"
235,Loaded_Language,"The idea of transubstantiation struck me as I read that document as possibly the strangest and most earth shattering thing Id ever heard. It shocked me out of a kind of swamp of intellectual worldliness; presenting the idea to my thirsty mind that there could indeed be fantastic realities far more wonderful than the banal and painfully uninteresting secularist worldview I had been taught to accept. It was as though someone had plausibly told me that, yes, there were fairies and magical kingdoms in real life, just around the corner."
1451,Doubt,"Reading Obamas 1995 memoir, you might almost get the impression that after a prudent first term, during his second he might side with, I dunno, Black Lives Matter and encourage a wave of black rage and police retreat that drove up the death toll from murder by 20% in his last two years in office, an incremental death toll a little bigger than the U.S. combat death toll from the equally stupid Iraq War"
2243,Appeal_to_Authority,"I personally know [Farrakhan], Ive been to his home, done meetings, participated in events with him, Davis told TheDC. I dont regard Louis Farrakhan as an aberration or anything, I regard him as an outstanding human being who commands a following of individuals who are learned and articulate and he plays a big role in the lives of thousands and thousands and thousands and thousands of people, he emphasized later."
2281,Causal_Oversimplification,"The FBIs bafflement here is part of its deep, deep corruption. The FBI doesnt acknowledge that there is a global jihad, or that Islam has anything to do with terrorism. It doesnt admit that there is a war going on, and treats each act of Islamic terror as if it were a separate and discrete criminal event, unrelated to all the others"
3220,Doubt,"Another problem with Farages statement is that it manifests a remarkable ignorance of history. While he is deeply concerned that British people not begin to think that resisting jihad terror means that they are in a battle with the entire religion of Islam, he appears unaware of the fact that many Muslims throughout history have considered their entire religion to be at war with the entire non-Muslim world"
3226,Appeal_to_Authority,"Fatimid caliph Al-Amir bi-Ahkamillah issued this edict: Now, the prior degradation of the infidels in this world before the life to comewhere it is their lotis considered an act of piety; and the imposition of their poll tax [jizya], until they pay the tribute out of hand and have been humbled (Koran 9:29) is a divinely ordained obligation .The dhimmis payment of his dues by a bill drawn on a Muslim, or by delegating a real believer to pay it in his name will not be tolerated. It must be exacted from him directly in order to vilify and humiliate him, so that Islam and its people may be exalted and the race of infidels brought low. The jizya is to be imposed on all of them in full, without exception"
3227,Appeal_to_Authority,"the Umayyad caliph Umar ibn Abd al-Aziz sent out a message to the governors of the various Islamic provinces: O you who believe! The non-Muslims are nothing but dirt. Allah has created them to be partisans of Satan; most treacherous in regard to all they do; whose whole endeavor in this nether life is useless, though they themselves imagine that they are doing fine work. Upon them rests the curse of Allah, of the Angels and of man collectively"
4029,Appeal_to_fear-prejudice,"Ghost guns are already a problem; they are used not just by lone shooters

The longest sequences consist of multiple connected sentences and/or quotes with somewhat arbitrary cut-off points.

# SI (Task 1)

## Sentences as split by NLTK

In [7]:
# The quoting=3 part is really important!

df = pd.read_csv(SI_LABELS_FILE, sep='\t', names=['doc_id', 'sent_id', 'idx_start', 'idx_end', 'tokens', 'label'], encoding='utf-8', quoting=3)
df = df.drop(columns=['doc_id', 'idx_start', 'idx_end', 'label'])
df = df.groupby('sent_id')['tokens'].apply(list).to_frame()
df['sent_len'] = df['tokens'].apply(lambda x : len(x))
pd.set_option('display.max_colwidth', -1)
df.head(15)


,tokens,sent_len
sent_id,,
1,"[Next, plague, outbreak, in, Madagascar, could, be, ', stronger, ', :, WHO, , Geneva, -, The, World, Health, Organisation, chief, on, Wednesday, said, a, deadly, plague, epidemic, appeared, to, have, been, brought, under, control, in, Madagascar, ,, but, warned, the, next, outbreak, would, likely, be, stronger, .]",47
2,"["", The, next, transmission, could, be, more, pronounced, or, stronger, ,, "", WHO, Director, -, General, Tedros, Adhanom, Ghebreyesus, told, reporters, in, Geneva, ,, insisting, that, "", the, issue, is, serious, ., ""]",33
3,"[An, outbreak, of, both, bubonic, plague, ,, which, is, spread, by, infected, rats, via, flea, bites, ,, and, pneumonic, plague, ,, spread, person, to, person, ,, has, killed, more, than, 200, people, in, the, Indian, Ocean, island, nation, since, August, .]",41
4,"[Madagascar, has, suffered, bubonic, plague, outbreaks, almost, every, year, since, 1980, ,, often, caused, by, rats, fleeing, forest, fires, .]",20
5,"[The, disease, tends, to, make, a, comeback, each, hot, rainy, season, ,, from, September, to, April, .]",17
6,"[On, average, ,, between, 300, and, 600, infections, are, recorded, every, year, among, a, population, approaching, 25, million, people, ,, according, to, a, UN, estimate, .]",26
7,"[But, Tedros, voiced, alarm, that, "", plague, in, Madagascar, behaved, in, a, very, ,, very, different, way, this, year, ., ""]",21
8,"[Cases, sprang, up, far, earlier, than, usual, and, ,, instead, of, being, confined, to, the, countryside, ,, the, disease, infiltrated, towns, .]",22
9,"[The, authorities, recorded, more, than, 2, 000, cases, ,, and, Tedros, said, Wednesday, the, death, toll, stood, at, 207, .]",20


## Baseline
Sentences split into fragments if they are longer than 35 tokens

In [9]:
df_baseline = pd.read_csv(SI_LABELS_FILE_BASELINE, sep='\t', names=['doc_id', 'sent_id', 'idx_start', 'idx_end', 'tokens', 'label'], encoding='utf-8', quoting=3)
df_baseline = df_baseline.drop(columns=['doc_id', 'idx_start', 'idx_end', 'label'])
df_baseline = df_baseline.groupby('sent_id')['tokens'].apply(list).to_frame()
df_baseline['sent_len'] = df_baseline['tokens'].apply(lambda x : len(x))
df_baseline.head(18)

,tokens,sent_len
sent_id,,
1,"[Next, plague, outbreak, in, Madagascar, could, be, ', stronger, ', :, WHO, , Geneva, -, The, World, Health, Organisation, chief, on, Wednesday, said, a, deadly, plague, epidemic, appeared, to, have, been, brought, under, control, in]",35
2,"[Madagascar, ,, but, warned, the, next, outbreak, would, likely, be, stronger, .]",12
3,"["", The, next, transmission, could, be, more, pronounced, or, stronger, ,, "", WHO, Director, -, General, Tedros, Adhanom, Ghebreyesus, told, reporters, in, Geneva, ,, insisting, that, "", the, issue, is, serious, ., ""]",33
4,"[An, outbreak, of, both, bubonic, plague, ,, which, is, spread, by, infected, rats, via, flea, bites, ,, and, pneumonic, plague, ,, spread, person, to, person, ,, has, killed, more, than, 200, people, in, the, Indian]",35
5,"[Ocean, island, nation, since, August, .]",6
6,"[Madagascar, has, suffered, bubonic, plague, outbreaks, almost, every, year, since, 1980, ,, often, caused, by, rats, fleeing, forest, fires, .]",20
7,"[The, disease, tends, to, make, a, comeback, each, hot, rainy, season, ,, from, September, to, April, .]",17
8,"[On, average, ,, between, 300, and, 600, infections, are, recorded, every, year, among, a, population, approaching, 25, million, people, ,, according, to, a, UN, estimate, .]",26
9,"[But, Tedros, voiced, alarm, that, "", plague, in, Madagascar, behaved, in, a, very, ,, very, different, way, this, year, ., ""]",21


## Improved splitting

- NLTK sentence splitter
- always split along linebreaks
- if a sentence is too long: split along quotes, semicolons, colons, commas

In [17]:
df_improved = pd.read_csv(SI_LABELS_FILE_IMPROVED, sep='\t', names=['doc_id', 'sent_id', 'idx_start', 'idx_end', 'tokens', 'label'], encoding='utf-8', quoting=3)
df_improved = df_improved.drop(columns=['doc_id', 'idx_start', 'idx_end', 'label'])
df_improved = df_improved.groupby('sent_id')['tokens'].apply(list).to_frame()
df_improved['sent_len'] = df_improved['tokens'].apply(lambda x : len(x))
df_improved.head(19)

,tokens,sent_len
sent_id,,
1,"[Next, plague, outbreak, in, Madagascar, could, be, ', stronger, ', :, WHO]",12
2,"[Geneva, -, The, World, Health, Organisation, chief, on, Wednesday, said, a, deadly, plague, epidemic, appeared, to, have, been, brought, under, control, in, Madagascar, ,, but, warned, the, next, outbreak, would, likely, be, stronger, .]",34
3,"["", The, next, transmission, could, be, more, pronounced, or, stronger, ,, "", WHO, Director, -, General, Tedros, Adhanom, Ghebreyesus, told, reporters, in, Geneva, ,, insisting, that, "", the, issue, is, serious, ., ""]",33
4,"[An, outbreak, of, both, bubonic, plague, ,, which, is, spread, by, infected, rats, via, flea, bites, ,, and, pneumonic, plague, ,, spread, person, to, person, ,]",26
5,"[has, killed, more, than, 200, people, in, the, Indian, Ocean, island, nation, since, August, ., ,]",16
6,"[Madagascar, has, suffered, bubonic, plague, outbreaks, almost, every, year, since, 1980, ,, often, caused, by, rats, fleeing, forest, fires, .]",20
7,"[The, disease, tends, to, make, a, comeback, each, hot, rainy, season, ,, from, September, to, April, .]",17
8,"[On, average, ,, between, 300, and, 600, infections, are, recorded, every, year, among, a, population, approaching, 25, million, people, ,, according, to, a, UN, estimate, .]",26
9,"[But, Tedros, voiced, alarm, that, "", plague, in, Madagascar, behaved, in, a, very, ,, very, different, way, this, year, ., ""]",21


In [18]:
pd.concat([df.describe(), df_baseline.describe(), df_improved.describe()], axis=1, sort=False)

,sent_len,sent_len,sent_len
count,15664.000000,19538.000000,21611.000000
mean,25.655005,20.564950,18.688538
std,16.519007,10.955816,9.426696
min,1.000000,1.000000,1.000000
25%,14.000000,11.000000,11.000000
50%,22.000000,20.000000,18.000000
75%,34.000000,32.000000,27.000000
max,173.000000,35.000000,35.000000


Back to the original input...

In [19]:
# df = df.sort_values(by=['sent_len'], ascending=False)
# df[df.sent_len > 60][['sent_len', 'tokens']]

- Punctuation marks add a lot of tokens to a sentence!
- Embedded quotes can be a problem
- Headlines w/o punctuation are recognized as part of the following sentence
- Some sentences are just very long (writing style)

Abbreviations are the opposite problem:

In [20]:
# df[df.sent_len == 1][['sent_len', 'tokens']]
df_improved[df_improved.sent_len == 1][['sent_len', 'tokens']]

,sent_len,tokens
sent_id,,
112,1,[SPONSORED]
489,1,[J.B.]
776,1,[A.]
1477,1,[–Newsweek]
2078,1,[and]
3705,1,[https://youtu.be/_DFeri21hlc]
4027,1,[Bryant]
4193,1,[St.]
4202,1,[Parolin]


Also:
- Punctuation marks that don't belong to the text and are analyzed as individual sentences
